<a href="https://colab.research.google.com/github/tarumi283/tarumi/blob/main/Cosinor%20for%20fitting%20data_24well_version1.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip -q install numpy scipy

In [43]:
from google.colab import files
files.upload()

print("アップロード完了。次のセルで INPUT_PATH を書いてください（ファイル名だけでOK）。")


Saving 2025-0826-1048_Plate1_F0_N_D.csv to 2025-0826-1048_Plate1_F0_N_D (1).csv
アップロード完了。次のセルで INPUT_PATH を書いてください（ファイル名だけでOK）。


In [80]:
from pathlib import Path

# ★アップロードしたファイル名（例）
INPUT_PATH = "/content/2025-0826-1048_Plate1_F0_N_D.csv"

# period scan settings
PERIOD_START = 20
PERIOD_END   = 30
PERIOD_STEP  = 0.1

# plate CSV settings
ANALYZE_ALL_WELLS = True   # True: 全well解析
WELL_NAME = "A1"           # ANALYZE_ALL_WELLS=Falseの時だけ有効
EXCLUDE_BACKGROUND = True

# output
OUTPUT_DIR = "cosinor_results"

# ---- check
p = Path(INPUT_PATH)
print("INPUT_PATH:", p)
print("Exists?:", p.exists())
if not p.exists():
    raise FileNotFoundError(
        "INPUT_PATH が見つかりません。アップロードしたファイル名と完全一致しているか確認してください。"
    )

USE_BINNING = True          # True: binningしてから解析 / False: 生データのまま解析
BIN_SIZE_H  = 0.1666667     # 10 min = 10/60 = 0.1666667 hours（例）
BIN_START   = 0             # 使い始めるbin番号（0始まり）
BIN_END     = 288          # 使い終わるbin番号（Noneなら最後まで）
# 例：最初の24時間を捨てたい(10分間隔)なら BIN_START = 24*6 = 144


INPUT_PATH: /content/2025-0826-1048_Plate1_F0_N_D.csv
Exists?: True


In [81]:
# ===== Cell 4: Cosinor core（EXE寄せ + acrophaseは“負表記” + PeakTimeも一貫） =====
# - モデル当てはめ： y = b0 + bc*cos(wt) + bs*sin(wt)
# - EXE寄せの位相定義： y = M + A*cos(wt - phi)
#     bc = A*cos(phi), bs = A*sin(phi), phi = atan2(bs, bc)
# - acrophase（表示用）は 0..360 を必ず負側で表示：phi_display = - (phi mod 360)（0は0）
# - PeakTime（実際のピーク時刻）は、内部の phi(0..360) を使って計算：
#     t_peak = (phi_mod360 / 360) * period
#   ※表示は負でも計算は正しいピーク位置になります

from __future__ import annotations

import math
from dataclasses import dataclass
from typing import List, Tuple

import numpy as np
from scipy.stats import f as f_dist


@dataclass
class PeriodRow:
    period: float
    f_stat: float
    p_value: float
    mesor: float
    amplitude: float
    acrophase_deg: float   # 表示用：0..360を負側にした値（例: -267）


def _fit_cosinor(t: np.ndarray, y: np.ndarray, period: float) -> Tuple[float, float, float, float, float]:
    """
    Fit:
      y = b0 + bc*cos(wt) + bs*sin(wt)
    Return: b0, bc, bs, SSE_full, SSE_null
    """
    w = 2.0 * np.pi / period
    X = np.column_stack([np.ones_like(t), np.cos(w * t), np.sin(w * t)])
    beta, *_ = np.linalg.lstsq(X, y, rcond=None)

    y_hat = X @ beta
    resid = y - y_hat
    sse_full = float(np.sum(resid ** 2))

    y_mean = float(np.mean(y))
    sse_null = float(np.sum((y - y_mean) ** 2))

    return float(beta[0]), float(beta[1]), float(beta[2]), sse_full, sse_null


def _f_test(sse_full: float, sse_null: float, n: int) -> Tuple[float, float]:
    """F-test for adding cos+sin (df_num=2) vs intercept only (df_den=n-3)."""
    if n <= 3:
        return float("nan"), float("nan")
    df_den = n - 3
    if sse_full <= 0:
        return float("inf"), 0.0

    f_stat = ((sse_null - sse_full) / 2.0) / (sse_full / df_den)
    if f_stat < 0:
        f_stat = 0.0
    p = float(f_dist.sf(f_stat, 2, df_den))
    return float(f_stat), p


def _phi_deg_mod360_from_bcbs(bc: float, bs: float) -> float:
    """
    EXE寄せ:
      y = M + A*cos(wt - phi)
      bc = A*cos(phi), bs = A*sin(phi)
      phi = atan2(bs, bc)  (radians)
    Return: phi in degrees wrapped into [0, 360).
    """
    phi = math.atan2(bs, bc)  # radians
    deg = (phi * 180.0 / math.pi) % 360.0
    return float(deg)


def _amp_acrophase_display_neg(bc: float, bs: float) -> Tuple[float, float]:
    """
    amplitude と “表示用acrophase（負表記）” を返す。
      phi_mod360 in [0,360)
      display = 0なら0、それ以外は -phi_mod360
    """
    amp = math.sqrt(bc * bc + bs * bs)
    phi_mod360 = _phi_deg_mod360_from_bcbs(bc, bs)
    if phi_mod360 == 0.0:
        display = 0.0
    else:
        display = -phi_mod360
    return float(amp), float(display)


def cosinor_periodogram_exe_style(
    t: np.ndarray,
    y: np.ndarray,
    period_start: float,
    period_end: float,
    period_step: float,
) -> Tuple[PeriodRow, List[PeriodRow]]:
    """Return (best_row, all_rows). best_row is the row with minimum p_value."""
    t = np.asarray(t, float)
    y = np.asarray(y, float)
    if t.shape != y.shape:
        raise ValueError("t and y must have the same shape.")
    if period_step <= 0:
        raise ValueError("period_step must be > 0")
    if period_end <= period_start:
        raise ValueError("period_end must be > period_start")

    periods = np.arange(period_start, period_end + 1e-12, period_step, dtype=float)
    n = int(t.size)

    rows: List[PeriodRow] = []
    for P in periods:
        b0, bc, bs, sse_full, sse_null = _fit_cosinor(t, y, float(P))
        f_stat, p = _f_test(sse_full, sse_null, n=n)
        amp, aph_display = _amp_acrophase_display_neg(bc, bs)

        rows.append(
            PeriodRow(
                period=float(P),
                f_stat=f_stat,
                p_value=p,
                mesor=b0,
                amplitude=amp,
                acrophase_deg=aph_display,  # 表示用（負）
            )
        )

    best = min(rows, key=lambda r: (r.p_value, -r.f_stat))
    return best, rows


def acrophase_to_peak_time(acrophase_deg: float, period_h: float) -> float:
    """
    PeakTime は「内部のphi(0..360)」で計算するのが正解。
    しかし acrophase_deg は“表示用の負表記”なので、まず phi_mod360 に戻す：

      if acrophase_deg == 0: phi_mod360=0
      else: phi_mod360 = (-acrophase_deg)  （例: -267 -> 267）

    そして EXE寄せの式：
      t_peak = (phi_mod360 / 360) * period
    を [0, period) に wrap して返す。
    """
    if acrophase_deg == 0.0:
        phi_mod360 = 0.0
    else:
        phi_mod360 = (-acrophase_deg) % 360.0

    t_peak = (phi_mod360 / 360.0) * period_h
    return float(t_peak % period_h)


In [82]:
import csv
import re
from typing import Dict, Tuple, Optional

import numpy as np
from pathlib import Path


def looks_like_plate_reader_csv(path: Path) -> bool:
    """Heuristic sniff for plate-reader CSV exports."""
    try:
        with path.open("r", encoding="utf-8", errors="ignore") as f:
            head = "".join([next(f) for _ in range(30)])
        return ("Time(h)" in head) and ("RLU" in head)
    except Exception:
        return False


def parse_plate_reader_csv(path: Path) -> Tuple[Dict[str, Tuple[np.ndarray, np.ndarray]], Dict[str, str]]:
    """
    Returns:
      series_by_well: {well: (t_hours, y_rlu)}
      meta: metadata
    """
    lines = path.read_text(encoding="utf-8", errors="ignore").splitlines()

    meta: Dict[str, str] = {}
    header_idx: Optional[int] = None

    # find header row and collect meta
    for i, ln in enumerate(lines[:200]):
        if ln.startswith('"Date","Time(h)"') or ln.startswith("Date,Time(h)") or ("Time(h)" in ln and "RLU" in ln and "Date" in ln):
            header_idx = i
            break
        m = re.match(r'^"([^"]+)"\s*,\s*(.*)$', ln)
        if m:
            meta[m.group(1).strip()] = m.group(2).strip().strip('"')

    if header_idx is None:
        raise ValueError("Could not find header row containing Date/Time(h)/RLU")

    # detect well-label row near header
    well_labels: list[str] = []
    well_pat = re.compile(r"^[A-H]\d+$")

    for back in range(max(0, header_idx - 12), header_idx):
        ln = lines[back]
        # require at least one typical well token
        if not any(w in ln for w in ("A1", "A2", "B1", "C1", "D1")):
            continue
        try:
            row = next(csv.reader([ln]))
        except Exception:
            continue
        cand = [c.strip().strip('"') for c in row if c.strip().strip('"')]
        if any(well_pat.fullmatch(c) for c in cand):
            well_labels = [c for c in cand if (c == "Background" or well_pat.fullmatch(c))]
            break

    # read from header row onward using csv.reader
    rows: list[list[str]] = []
    with path.open("r", encoding="utf-8", errors="ignore", newline="") as f:
        reader = csv.reader(f)
        for idx, row in enumerate(reader):
            if idx < header_idx:
                continue
            rows.append(row)

    header = rows[0]
    data_rows = rows[1:]

    def clean(s: str) -> str:
        return s.strip().strip('"')

    time_cols = [i for i, c in enumerate(header) if clean(c) == "Time(h)"]
    rlu_cols = [i for i, c in enumerate(header) if clean(c) == "RLU"]
    if not time_cols or not rlu_cols or len(time_cols) != len(rlu_cols):
        raise ValueError("Unexpected header: cannot find paired Time(h)/RLU columns.")

    pairs = list(zip(time_cols, rlu_cols))

    # assign names
    if well_labels and len(well_labels) >= len(pairs):
        names = well_labels[: len(pairs)]
    else:
        names = ["Background"] + [f"Well{i}" for i in range(1, len(pairs))]

    series_by_well: Dict[str, Tuple[np.ndarray, np.ndarray]] = {}
    for name, (ti, yi) in zip(names, pairs):
        t_vals: list[float] = []
        y_vals: list[float] = []
        for r in data_rows:
            if len(r) <= max(ti, yi):
                continue
            ts = r[ti].strip()
            ys = r[yi].strip()
            if ts == "" or ys == "":
                continue
            try:
                t_vals.append(float(ts))
                y_vals.append(float(ys))
            except ValueError:
                continue

        if len(t_vals) >= 4:
            series_by_well[name] = (np.asarray(t_vals, float), np.asarray(y_vals, float))

    if not series_by_well:
        raise ValueError("No valid series extracted from CSV.")
    return series_by_well, meta


In [83]:
import numpy as np

def bin_time_series(t, y, bin_size_h: float, reducer: str = "mean"):
    """
    t: hours（float）
    y: signal
    bin_size_h: bin width (hours)
    reducer: "mean" or "median"
    Returns: (t_binned, y_binned) where t_binned is bin-center time.
    """
    t = np.asarray(t, float)
    y = np.asarray(y, float)
    if len(t) != len(y):
        raise ValueError("t and y length mismatch")
    if bin_size_h <= 0:
        raise ValueError("bin_size_h must be > 0")

    # sort by time just in case
    idx = np.argsort(t)
    t = t[idx]
    y = y[idx]

    t0 = t[0]
    # bin index for each sample
    b = np.floor((t - t0) / bin_size_h).astype(int)

    nb = b.max() + 1
    t_out = np.empty(nb, float)
    y_out = np.empty(nb, float)

    for k in range(nb):
        m = (b == k)
        if not np.any(m):
            t_out[k] = np.nan
            y_out[k] = np.nan
            continue
        # bin center time
        t_out[k] = t0 + (k + 0.5) * bin_size_h
        vals = y[m]
        if reducer == "median":
            y_out[k] = np.nanmedian(vals)
        else:
            y_out[k] = np.nanmean(vals)

    # drop NaN bins (should not happen usually)
    ok = ~np.isnan(t_out) & ~np.isnan(y_out)
    return t_out[ok], y_out[ok]


def select_bins(t_b, y_b, start_bin: int = 0, end_bin: int | None = None):
    """
    start_bin: 0-based inclusive
    end_bin: 0-based exclusive (Python slice). None means to the end.
    """
    if start_bin < 0:
        raise ValueError("BIN_START must be >= 0")
    n = len(y_b)
    if end_bin is None:
        end_bin = n
    if end_bin <= start_bin:
        raise ValueError("BIN_END must be > BIN_START (or None)")
    end_bin = min(end_bin, n)
    return t_b[start_bin:end_bin], y_b[start_bin:end_bin]


In [84]:
# ===== Cell 7: TXT report only (fixed) =====

import re
from pathlib import Path
from typing import List, Optional

def safe_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_\-\.]+", "_", name.strip()) or "output"

def format_exe_report(
    best: PeriodRow,
    rows: List[PeriodRow],
    title: str,
    peak_time_h: float,
    meta: Optional[dict] = None
) -> str:
    out: List[str] = []
    out.append(title)
    if meta:
        for k in ("Plate format", "Date", "Time", "Integral time", "Interval time", "Measurement time", "Background"):
            if k in meta:
                out.append(f"{k}: {meta[k]}")
        out.append("")

    out.append("Cosinor (best period)")
    out.append(f"Best period {best.period:.6g} h")
    out.append(f"MESOR      {best.mesor:.6g}")
    out.append(f"Amplitude  {best.amplitude:.6g}")
    out.append(f"Acrophase  {best.acrophase_deg:.6g} deg")
    out.append(f"Peak time  {peak_time_h:.6g} h")
    out.append(f"F          {best.f_stat:.6g}")
    out.append(f"p          {best.p_value:.6g}")
    out.append("")
    out.append("Periodogram")
    out.append("Period\tF\tp")
    for r in rows:
        out.append(f"{r.period:.6g}\t{r.f_stat:.6g}\t{r.p_value:.6g}")
    return "\n".join(out) + "\n"


# ---- run
inp = Path(INPUT_PATH)
outdir = Path(OUTPUT_DIR)
outdir.mkdir(parents=True, exist_ok=True)

written = []

if inp.suffix.lower() == ".csv" and looks_like_plate_reader_csv(inp):
    series_by_well, meta = parse_plate_reader_csv(inp)

    if ANALYZE_ALL_WELLS:
        chosen = dict(series_by_well)
        if EXCLUDE_BACKGROUND:
            chosen.pop("Background", None)
    else:
        lut = {k.lower(): k for k in series_by_well.keys()}
        if WELL_NAME.lower() not in lut:
            raise KeyError(f"WELL_NAME '{WELL_NAME}' not found. Available: {', '.join(series_by_well.keys())}")
        key = lut[WELL_NAME.lower()]
        chosen = {key: series_by_well[key]}

    step = PERIOD_STEP  # ★ここが正しい（セル3で設定した値を使う）

    for well, (t, y) in chosen.items():
        if USE_BINNING:
            t2, y2 = bin_time_series(t, y, BIN_SIZE_H, reducer="mean")
            t2, y2 = select_bins(t2, y2, BIN_START, BIN_END)
        else:
            t2, y2 = t, y

        best, rows = cosinor_periodogram_exe_style(t2, y2, PERIOD_START, PERIOD_END, step)
        peak_time_h = acrophase_to_peak_time(best.acrophase_deg, best.period)

        title = f"Cosinor Periodogram Result  ({inp.name} / {well})"
        rep = format_exe_report(best, rows, title, peak_time_h, meta=meta)

        out_path = outdir / f"{safe_filename(well)}.txt"
        out_path.write_text(rep, encoding="utf-8")
        written.append(str(out_path))

else:
    raise ValueError("この版は plate reader CSV（Time(h)/RLU のワイドCSV）向けです。")

print("[OK] wrote:")
for p in written[:30]:
    print(" -", p)
if len(written) > 30:
    print(f" ... ({len(written)} files) total")
print("Output dir:", outdir.resolve())



[OK] wrote:
 - cosinor_results/A1.txt
 - cosinor_results/A2.txt
 - cosinor_results/A3.txt
 - cosinor_results/A4.txt
 - cosinor_results/A5.txt
 - cosinor_results/A6.txt
 - cosinor_results/B1.txt
 - cosinor_results/B2.txt
 - cosinor_results/B3.txt
 - cosinor_results/B4.txt
 - cosinor_results/B5.txt
 - cosinor_results/B6.txt
 - cosinor_results/C1.txt
 - cosinor_results/C2.txt
 - cosinor_results/C3.txt
 - cosinor_results/C4.txt
 - cosinor_results/C5.txt
 - cosinor_results/C6.txt
 - cosinor_results/D1.txt
 - cosinor_results/D2.txt
 - cosinor_results/D3.txt
 - cosinor_results/D4.txt
 - cosinor_results/D5.txt
 - cosinor_results/D6.txt
Output dir: /content/cosinor_results


In [85]:
# ===== Cell 8 (FULL): TXT + Excel/CSV + DEBUG (確実にエラーが出にくい版) =====

# Colab推奨（!pip より %pip の方がカーネルに確実に入る）
%pip -q install pandas openpyxl

import re
import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Optional

# -------- utilities --------
def safe_filename(name: str) -> str:
    return re.sub(r"[^A-Za-z0-9_\-\.]+", "_", str(name).strip()) or "output"

def format_exe_report(
    best: PeriodRow,
    rows: List[PeriodRow],
    title: str,
    peak_time_h: float,
    meta: Optional[dict] = None
) -> str:
    out: List[str] = []
    out.append(title)
    if meta:
        for k in ("Plate format", "Date", "Time", "Integral time", "Interval time", "Measurement time", "Background"):
            if k in meta:
                out.append(f"{k}: {meta[k]}")
        out.append("")

    out.append("Cosinor (best period)")
    out.append(f"Best period {best.period:.6g} h")
    out.append(f"MESOR      {best.mesor:.6g}")
    out.append(f"Amplitude  {best.amplitude:.6g}")
    out.append(f"Acrophase  {best.acrophase_deg:.6g} deg")
    out.append(f"Peak time  {peak_time_h:.6g} h")
    out.append(f"F          {best.f_stat:.6g}")
    out.append(f"p          {best.p_value:.6g}")
    out.append("")
    out.append("Periodogram")
    out.append("Period\tF\tp")
    for r in rows:
        out.append(f"{r.period:.6g}\t{r.f_stat:.6g}\t{r.p_value:.6g}")
    return "\n".join(out) + "\n"


# ====== ここから実行 ======
inp = Path(INPUT_PATH)
outdir = Path(OUTPUT_DIR)
outdir.mkdir(parents=True, exist_ok=True)

written_txt = []
summary_rows = []

# --- 最初に最低限の前提チェック（ここで落ちるならセルの実行順が問題） ---
required = [
    "looks_like_plate_reader_csv",
    "parse_plate_reader_csv",
    "bin_time_series",
    "select_bins",
    "cosinor_periodogram_exe_style",
    "acrophase_to_peak_time",
    "_fit_cosinor",          # デバッグで使う
]
missing = [name for name in required if name not in globals()]
if missing:
    raise RuntimeError(
        "前のセルが実行されていません。足りない関数: " + ", ".join(missing)
    )

# --- CSV 判定＆読み込み ---
if not (inp.suffix.lower() == ".csv" and looks_like_plate_reader_csv(inp)):
    raise ValueError("この版は plate reader CSV（Time(h)/RLU のワイドCSV）向けです。")

series_by_well, meta = parse_plate_reader_csv(inp)

print("=== Wells (first 30) ===")
print(list(series_by_well.keys())[:30])

# --- well選択 ---
if ANALYZE_ALL_WELLS:
    chosen = dict(series_by_well)
    if EXCLUDE_BACKGROUND:
        chosen.pop("Background", None)
else:
    lut = {k.lower(): k for k in series_by_well.keys()}
    if WELL_NAME.lower() not in lut:
        raise KeyError(f"WELL_NAME '{WELL_NAME}' not found. Available: {', '.join(series_by_well.keys())}")
    key = lut[WELL_NAME.lower()]
    chosen = {key: series_by_well[key]}

# ★デバッグ対象well（A1があればA1、なければ先頭）
DEBUG_WELL = "A1" if "A1" in chosen else list(chosen.keys())[0]
print("DEBUG_WELL =", DEBUG_WELL)

step = PERIOD_STEP  # セル3で設定した period step をそのまま使う

# --- 解析ループ ---
for well, (t, y) in chosen.items():
    t = np.asarray(t, float)
    y = np.asarray(y, float)

    # --- binning ---
    if USE_BINNING:
        t2, y2 = bin_time_series(t, y, BIN_SIZE_H, reducer="mean")
        t2, y2 = select_bins(t2, y2, BIN_START, BIN_END)
    else:
        t2, y2 = t, y

    # --- デバッグ出力（1 wellだけ） ---
    if well == DEBUG_WELL:
        print("\n=== DEBUG START ===")
        print("well:", well)
        print("raw n =", len(t), "t0 =", float(t[0]), "t_end =", float(t[-1]),
              "y_mean =", float(np.mean(y)), "y_min/max =", float(np.min(y)), float(np.max(y)))
        print("used n =", len(t2), "t2_0 =", float(t2[0]), "t2_end =", float(t2[-1]),
              "y2_mean =", float(np.mean(y2)), "y2_min/max =", float(np.min(y2)), float(np.max(y2)))

        # 固定周期(24h)で係数とSSEを出す（ここが変わるなら入力が別物）
        b0, bc, bs, sse_full, sse_null = _fit_cosinor(t2, y2, 24.0)
        print("P=24h fit:",
              "b0 =", b0, "bc =", bc, "bs =", bs,
              "SSE_full =", sse_full, "SSE_null =", sse_null)

        # 使っている時間点の先頭だけ見せる（ズレ確認）
        print("t2 first 10:", np.round(t2[:10], 6))
        print("y2 first 10:", np.round(y2[:10], 6))
        print("=== DEBUG END ===\n")

    # --- cosinor scan ---
    best, rows = cosinor_periodogram_exe_style(t2, y2, PERIOD_START, PERIOD_END, step)
    peak_time_h = acrophase_to_peak_time(best.acrophase_deg, best.period)

    # --- txt report ---
    title = f"Cosinor Periodogram Result  ({inp.name} / {well})"
    rep = format_exe_report(best, rows, title, peak_time_h, meta=meta)
    out_path = outdir / f"{safe_filename(well)}.txt"
    out_path.write_text(rep, encoding="utf-8")
    written_txt.append(str(out_path))

    # --- summary row ---
    summary_rows.append({
        "Well": well,
        "BestPeriod_h": best.period,
        "MESOR": best.mesor,
        "Amplitude": best.amplitude,
        "Acrophase_deg": best.acrophase_deg,
        "PeakTime_h": peak_time_h,
        "F": best.f_stat,
        "p": best.p_value,
    })

# ---- write summary to Excel/CSV
df = pd.DataFrame(summary_rows)

# Wellが A1..H12 形式のときだけ整列（形式が違うと落ちるので保険）
m_row = df["Well"].astype(str).str.extract(r"^([A-H])")[0]
m_col = df["Well"].astype(str).str.extract(r"(\d+)$")[0]
if m_row.notna().all() and m_col.notna().all():
    df["__row"] = m_row
    df["__col"] = m_col.astype(int)
    df = df.sort_values(["__row", "__col"]).drop(columns=["__row", "__col"])

xlsx_path = outdir / "cosinor_summary.xlsx"
csv_path  = outdir / "cosinor_summary.csv"
df.to_excel(xlsx_path, index=False)
df.to_csv(csv_path, index=False)

print("[OK] wrote txt:")
for p in written_txt[:20]:
    print(" -", p)
if len(written_txt) > 20:
    print(f" ... ({len(written_txt)} files) total")

print("\n[OK] wrote summary:")
print(" -", xlsx_path)
print(" -", csv_path)

df


=== Wells (first 30) ===
['Background', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6']
DEBUG_WELL = A1

=== DEBUG START ===
well: A1
raw n = 423 t0 = 0.015833 t_end = 70.3475 y_mean = -24.872340425531913 y_min/max = -1184.0 1406.0
used n = 288 t2_0 = 0.09916635 t2_end = 47.93250925 y2_mean = 14.729166666666666 y2_min/max = -1184.0 1406.0
P=24h fit: b0 = 14.729224532735591 bc = -285.1178654202036 bs = -1011.0271909581938 SSE_full = 15112784.20482757 SSE_null = 174012174.875
t2 first 10: [0.099166 0.265833 0.4325   0.599166 0.765833 0.9325   1.099167 1.265833
 1.4325   1.599167]
y2 first 10: [-107. -132. -147. -164. -179. -195. -213. -231. -250. -270.]
=== DEBUG END ===

[OK] wrote txt:
 - cosinor_results/A1.txt
 - cosinor_results/A2.txt
 - cosinor_results/A3.txt
 - cosinor_results/A4.txt
 - cosinor_results/A5.txt
 - cosinor_results/A6.txt
 - cosinor_results/B1.txt
 - cosinor_results/B2.txt
 

,Well,BestPeriod_h,MESOR,Amplitude,Acrophase_deg,PeakTime_h,F,p
0,A1,23.6,19.631061,1059.812870,-260.201585,17.057659,1562.871285,2.423362e-154
1,A2,23.0,31.038728,1021.178342,-269.712911,17.231658,1201.351039,1.344565e-139
2,A3,23.2,26.700553,1076.698494,-262.780627,16.934752,1190.874618,4.101399e-139
3,A4,23.2,29.309156,1023.710022,-265.680602,17.121639,1157.643573,1.495740e-137
4,A5,23.7,17.102918,998.039278,-257.379928,16.944179,1427.700844,3.126088e-149
5,A6,23.8,20.986171,1075.761946,-259.976816,17.187356,1737.895816,2.178193e-160
6,B1,23.7,128.678832,1217.921493,-261.661021,17.226017,792.630951,3.709341e-117
7,B2,22.1,114.818637,959.446881,-286.646805,17.596929,476.184864,1.362121e-91
8,B3,22.3,118.020480,1000.409773,-277.875179,17.212824,472.971272,2.860939e-91
9,B4,22.1,97.963078,895.747384,-284.204719,17.447012,396.181504,5.055907e-83


In [79]:

from google.colab import files

files.download(f"{OUTPUT_DIR}/cosinor_summary.xlsx")
# ついでにCSVも欲しければ:
# files.download(f"{OUTPUT_DIR}/cosinor_summary.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>